In [1]:
import os
import json
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import numpy as np
from seqeval.metrics import classification_report, accuracy_score
import torch
import torch.nn as nn

In [2]:
MAX_LEN = 512  # BERT's max sequence length
DATA_DIR = "/media/smartdragon/WORK/6th Semester/22AIE315 - Natural Language Processing/Project/New_Json_Files"

In [3]:
def load_jsonl(filepath):
    data = []
    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            data.append(json.loads(line.strip()))
    return Dataset.from_list(data)


In [4]:
def split_long_sentence(tokens, tags, max_len=MAX_LEN - 2):  # -2 for [CLS] and [SEP]
    chunks = []
    for i in range(0, len(tokens), max_len):
        chunk_tokens = tokens[i:i+max_len]
        chunk_tags = tags[i:i+max_len]
        chunks.append({'tokens': chunk_tokens, 'tags': chunk_tags})
    return chunks

In [5]:
def preprocess_dataset(dataset):
    split_data = []
    for example in dataset:
        split_sentences = split_long_sentence(example['tokens'], example['tags'])
        split_data.extend(split_sentences)
    return Dataset.from_list(split_data)

In [6]:
def tokenize_and_align_labels(example, tokenizer, label2id):
    tokenized = tokenizer(example['tokens'], truncation=True, max_length=512, is_split_into_words=True)
    word_ids = tokenized.word_ids()

    aligned_labels = []
    previous_word_idx = None
    for word_idx in word_ids:
        if word_idx is None:
            aligned_labels.append(-100)
        elif word_idx != previous_word_idx:
            aligned_labels.append(label2id.get(example['tags'][word_idx], -100))
        else:
            aligned_labels.append(label2id.get(example['tags'][word_idx], -100))

        previous_word_idx = word_idx

    tokenized['labels'] = aligned_labels
    return tokenized

In [7]:
def compute_metrics(p, id2label):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = []
    true_predictions = []

    for i in range(len(labels)):
        label_sequence = []
        prediction_sequence = []

        for j in range(len(labels[i])):
            if labels[i][j] != -100:  # Exclude padding tokens
                label_sequence.append(id2label.get(labels[i][j], "O"))
                prediction_sequence.append(id2label.get(predictions[i][j], "O"))

        true_labels.append(label_sequence)
        true_predictions.append(prediction_sequence)

    report = classification_report(true_labels, true_predictions, output_dict=True)
    accuracy = accuracy_score(true_labels, true_predictions)

    return {
        "f1": report["micro avg"]["f1-score"],
        "precision": report["micro avg"]["precision"],
        "recall": report["micro avg"]["recall"],
        "accuracy": accuracy
    }

In [8]:
def detect_batches():
    batch_numbers = set()
    for file in os.listdir(DATA_DIR):
        if file.startswith("combined_train_") and file.endswith(".jsonl"):
            batch_num = file.split("_")[-1].replace(".jsonl", "")
            if batch_num.isdigit():
                batch_numbers.add(int(batch_num))
    return sorted(batch_numbers)

In [9]:
def train_and_evaluate(batch_num):
    dataset_prefix = f"combined_{batch_num}"

    train_file = os.path.join(DATA_DIR, f"combined_train_{batch_num}.jsonl")
    dev_file = os.path.join(DATA_DIR, f"combined_dev_{batch_num}.jsonl")
    test_file = os.path.join(DATA_DIR, f"combined_test_{batch_num}.jsonl")

    if not os.path.exists(train_file) or not os.path.exists(dev_file) or not os.path.exists(test_file):
        print(f"🚨 Skipping batch {batch_num}, files not found.")
        return

    print(f"🚀 Processing batch {batch_num}...")

    
    model_path = f"/media/smartdragon/Windows-SSD/Users/sriva/Documents/NLP/BioBERT/biobert-finetuned-batch{batch_num-1}"
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    # Load label2id from Batch 1 to ensure consistent label space
    with open(os.path.join(model_path, "label2id.json"), "r") as f:
        old_label2id = json.load(f)
    id2label = {v: k for k, v in old_label2id.items()}
    print(f"✅ Loaded label2id from Batch {batch_num - 1}: {old_label2id}")

    train_dataset = preprocess_dataset(load_jsonl(train_file))
    dev_dataset = preprocess_dataset(load_jsonl(dev_file))
    test_dataset = preprocess_dataset(load_jsonl(test_file))
    print(f"After splitting: Train={len(train_dataset)}, Dev={len(dev_dataset)}, Test={len(test_dataset)}")

    unique_tags = set(tag for example in train_dataset['tags'] for tag in example)
    label2id = {tag: i for i, tag in enumerate(sorted(unique_tags))}
    id2label = {v: k for k, v in label2id.items()}

    train_dataset = train_dataset.map(lambda x: tokenize_and_align_labels(x, tokenizer, label2id))
    dev_dataset = dev_dataset.map(lambda x: tokenize_and_align_labels(x, tokenizer, label2id))
    test_dataset = test_dataset.map(lambda x: tokenize_and_align_labels(x, tokenizer, label2id))

    # Load the number of labels from batch 1
    old_num_labels = len(old_label2id)  # Should match previous training setup
    new_num_labels = len(label2id)  # Ensure this matches the new dataset

    # Load the previously trained model from batch 1
    model = AutoModelForTokenClassification.from_pretrained(
        model_path, num_labels=old_num_labels
    )

    # Extract the old classifier layer
    old_classifier = model.classifier

    # Create a new classifier layer with updated label count
    new_classifier = nn.Linear(old_classifier.in_features, new_num_labels)

    # Transfer weights from the old classifier to the new one (for common labels)
    with torch.no_grad():
        num_common_labels = min(old_num_labels, new_num_labels)
        new_classifier.weight[:num_common_labels, :] = old_classifier.weight[:num_common_labels, :]
        new_classifier.bias[:num_common_labels] = old_classifier.bias[:num_common_labels]

    # Assign the updated classifier to the model
    model.classifier = new_classifier

    # Save updated model before continuing training
    model.save_pretrained("/media/smartdragon/Windows-SSD/Users/sriva/Documents/NLP/BioBERT/BioBERT-updated")
    print(f"✅ Model updated to support new label set while keeping batch {batch_num - 1} training.")

    model_checkpoint = f"/media/smartdragon/Windows-SSD/Users/sriva/Documents/NLP/BioBERT/biobert-finetuned-batch{batch_num}"
    # Create the directory if it does not exist
    os.makedirs(model_checkpoint, exist_ok=True)
    print(f"Directory ensured: {model_checkpoint}")
    model_path_2 = "/media/smartdragon/Windows-SSD/Users/sriva/Documents/NLP/BioBERT/BioBERT-updated"
    model = AutoModelForTokenClassification.from_pretrained(model_path_2, num_labels=len(label2id))

    training_args = TrainingArguments(
        output_dir="BioBERT-finetuned-mtl",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_dir="./logs",
        learning_rate=3e-5,
        per_device_train_batch_size=64,
        per_device_eval_batch_size=64,
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,
        push_to_hub=False,
        save_total_limit=2,
        gradient_accumulation_steps=8,  # This simulates batch size 4 * 4 = 16
        fp16=True,
    )

    data_collator = DataCollatorForTokenClassification(tokenizer)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=dev_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=lambda p: compute_metrics(p, id2label)
    )

    trainer.train()

    metrics = trainer.evaluate(test_dataset)
    print("Test Set Performance:")
    print(metrics)
    
    model.save_pretrained(model_checkpoint)
    tokenizer.save_pretrained(model_checkpoint)

    print(f"✅ Model trained and saved for batch {batch_num}")

    predictions, labels, _ = trainer.predict(test_dataset)
    predictions = np.argmax(predictions, axis=2)

        # Save label2id to file
    with open(f"{model_checkpoint}/label2id.json", "w") as f:
        json.dump(label2id, f)
    print("✅ Saved new_label2id mapping.")

    # Save label2id to file
    with open("label2id.json", "w") as f:
        json.dump(label2id, f)
    print("✅ Saved new_label2id mapping.")
    print("✅ Saved label2id mapping to model directory.")

    # results = classification_report(labels, predictions, output_dict=True)
    # with open(f"{model_checkpoint}_evaluation.json", "w") as f:
    #     json.dump(results, f)

    # print(f"📊 Evaluation results saved for batch {batch_num}")

In [10]:
batch_numbers = detect_batches()
print(f"📝 Detected batches: {batch_numbers}")

for batch_num in batch_numbers:
    train_and_evaluate(batch_num)

print("🎉 All detected batches processed successfully!")

📝 Detected batches: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
🚀 Processing batch 2...
✅ Loaded label2id from Batch 1: {'B-Anatomy': 0, 'I-Anatomy': 1, 'O': 2}
After splitting: Train=153823, Dev=58785, Test=99976


Map:   0%|          | 0/153823 [00:00<?, ? examples/s]

Map:   0%|          | 0/58785 [00:00<?, ? examples/s]

Map:   0%|          | 0/99976 [00:00<?, ? examples/s]

✅ Model updated to support new label set while keeping batch 1 training.
Directory ensured: /media/smartdragon/Windows-SSD/Users/sriva/Documents/NLP/BioBERT/biobert-finetuned-batch2


...

Map:   0%|          | 0/61643 [00:00<?, ? examples/s]

✅ Model updated to support new label set while keeping batch 19 training.
Directory ensured: /media/smartdragon/Windows-SSD/Users/sriva/Documents/NLP/BioBERT/biobert-finetuned-batch20


/media/smartdragon/Storage/anaconda3/envs/nlp/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_20214/2071289192.py:92: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,F1,Precision,Recall,Accuracy
1,No log,0.163144,0.720217,0.740934,0.700626,0.945105


Test Set Performance:
{'eval_loss': 0.163540318608284, 'eval_f1': 0.8083643253105046, 'eval_precision': 0.8118393234672304, 'eval_recall': 0.8049189491335942, 'eval_accuracy': 0.9633795545065644, 'eval_runtime': 10.0069, 'eval_samples_per_second': 6160.062, 'eval_steps_per_second': 96.334, 'epoch': 1.9882720875684128}
✅ Model trained and saved for batch 20
✅ Saved new_label2id mapping.
✅ Saved new_label2id mapping.
✅ Saved label2id mapping to model directory.
🎉 All detected batches processed successfully!
